# Información de BCRP

###### A continuación, vamos a extraer información del BCRP usando su api. Se realizará un proceso de automatización en el cual solo se tienen que elegir las variables que se desean obtener y seleccionar la fecha de interés a analizar.

In [1]:
# Códigos para actualizar el DashBoard Financiero

# PD04650MD : Reservas internacionales netas
# PD04692MD : Tasa de interés interbancaria S/.
# PD31893DD : Rendimiento del Bono del gobierno peruano a 10 años (en S/)
# PD31894DD : Rendimiento del Bono del gobierno peruano a 10 años (en US$)
# PD04637PD : TC Interbancario (S/ por US$) - Compra
# PD04638PD : TC Interbancario (S/ por US$) - Venta
# PD38026MD : Índice General Bursátil BVL (índice)

In [2]:
# Importamos las librerías necesarias para el trabajo
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from pandas_datareader import data
import requests

In [3]:
# Elegimos la lista que deseamos trabajar
# Dashboard financiero:
lista=["PD04650MD","PD04692MD","PD31893DD","PD31894DD","PD04637PD","PD04638PD","PD38026MD"] 

In [ ]:
# Este es un loop que me permite extraer todas las series de interés contenidas en "lista".
# Creamos primero un contenedor para colocar las series extraídas
d = {}

for serie in lista: 
    # A continuación, colocamos los datos requeridos para el uso del api
    formato = "json"
    fecha_inicio = "2020-10-01"
    fecha_fin = "2022-11-03"
    url = "https://estadisticas.bcrp.gob.pe/estadisticas/series/api/"+serie+"/"+formato+"/"+fecha_inicio+"/"+fecha_fin+"/"
    # Empezamos extrayendo la información en formato json   
    periodos = requests.get(url).json().get("periods")
    # VALORES
    dato = []
    for i in periodos:
        valores_list = i['values']
        for w in valores_list:
            w = float(w)
            dato.append(w)
            d[serie] = pd.DataFrame(dato)
    # FECHAS
    fechas = []
    for i in periodos:
        nombres = i['name']
        fechas.append(nombres)
        df_fechas = pd.DataFrame(fechas)

In [ ]:
# EXPORTAR DATOS: Con este código juntaremos todas las variables extraídas.
# Dashboard financiero
data = pd.concat((df_fechas, d['PD04650MD'], d['PD04692MD'], d['PD31893DD'], d['PD31894DD'], d['PD04637PD'], d['PD04638PD'], d['PD38026MD']), axis=1, ignore_index=True)
data.rename(columns = {0:'fecha', 1:'reservas_inter',2:'interbancaria',3:'bono_soles',4:'bono_dolares',5:'tc_compra',6:'tc_venta',7:'bvl'}, inplace = True)
# Exportamos a excel
data.to_excel('DB Financiero.xlsx', sheet_name='Monedas', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, startrow=0, startcol=0, engine=None, merge_cells=True, inf_rep='inf', freeze_panes=None, storage_options=None)